# Exploring Oura ring data

In [8]:
#%pip install pandas mlplot
 
import csv
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.pyplot import figure
from matplotlib.dates import DateFormatter


## First, let's grab data from local json file, using only sleep data

In [9]:
# Read in the CSV as a DataFrame
oura_df = pd.read_csv('data/oura_2022-01-10_2022-11-28_trends.csv')

# Print the first five rows of the DataFrame
# print(oura_df[0:5])
oura_df.head()

,date,Sleep Score,Total Sleep Score,REM Sleep Score,Deep Sleep Score,Sleep Efficiency Score,Restfulness Score,Sleep Latency Score,Sleep Timin Score,Total Sleep Duration,...,Long Periods of Inactivity,Readiness Score,Previous Night Score,Sleep Balance Score,Previous Day Activity Score,Activity Balance Score,Temperature Score,Resting Heart Rate Score,HRV Balance Score,Recovery Index Score
0,2022-01-13,67.0,71.0,91.0,57.0,58.0,77.0,64.0,35.0,24930.0,...,0,85.0,56,None,93,None,96.0,100.0,None,100.0
1,2022-01-14,78.0,100.0,98.0,58.0,76.0,73.0,56.0,31.0,34770.0,...,0,90.0,83,None,85,None,100.0,95.0,None,100.0
2,2022-01-15,76.0,71.0,77.0,96.0,96.0,84.0,97.0,16.0,24990.0,...,0,71.0,80,73,81,69,99.0,84.0,None,29.0
3,2022-01-16,75.0,64.0,87.0,75.0,90.0,90.0,75.0,57.0,19260.0,...,0,82.0,77,80,74,91,100.0,77.0,None,100.0
4,2022-01-17,78.0,75.0,83.0,68.0,90.0,88.0,97.0,47.0,25980.0,...,0,86.0,82,81,90,89,100.0,86.0,None,100.0


## Now, let's build an index of bedtime_end date components

In [10]:
# Reindex by bedtime end date
full_df['datetime'] = pd.to_datetime(full_df['bedtime_end'], utc=True).dt.normalize()
full_df['weekday'] = full_df['datetime'].dt.dayofweek
full_df['weekend'] = (full_df['datetime'].dt.dayofweek > 4).astype(int)
full_df['datetime_naive'] = full_df['datetime'].apply(lambda t: t.replace(tzinfo=None))
full_df = full_df.set_index('datetime_naive')
full_df.info()
full_df['weekend']

NameError: name 'full_df' is not defined

# So, how about some correlation plots?

In [ ]:
# Drop non-numerical column to make it neater, but use full range of data
df = full_df.drop('is_longest', axis=1) 

# calculate some extra numerical statistics for fun correlations
df['bedtime_start_hour'] = pd.to_datetime(df['bedtime_start'], utc=True).dt.hour
df['bedtime_end_hour'] = pd.to_datetime(df['bedtime_end'], utc=True).dt.hour

# Here is the fast and easy way to plot correlations
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')
corr.style.background_gradient(cmap='coolwarm').set_precision(2)


NameError: name 'full_df' is not defined

In [ ]:
# Here's another way (Create a plot)
f = plt.figure(figsize=(28, 18))
plt.matshow(df.corr(), fignum=f.number)
plt.xticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14, rotation=90)
plt.yticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);


NameError: name 'df' is not defined

<Figure size 2800x1800 with 0 Axes>

In [ ]:
# Or, just check one column and print all matches
corr = df.corr()['score'].sort_values()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(corr)

NameError: name 'df' is not defined

# Do my previous days metrics effect my next days sleep score?

In [15]:
df_next = df['score'] # Create another subset of data to compare
df_next = df_next.shift(periods=-1, axis=0, fill_value=0) # Shift back by one (compare days data with next days score)
df = df.join(df_next, rsuffix="_next") # Join to original df with a new name

NameError: name 'df' is not defined

In [14]:
# Here is the fast and easy way to plot correlations
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

NameError: name 'df' is not defined

# Does my daily activity effect my next night's sleep score?

In [13]:
full_activity_df = pd.DataFrame.from_dict(data['activity'])
# Index by summary_date aka day when activity period started, 4AM
full_activity_df['datetime'] = pd.to_datetime(full_activity_df['summary_date']).dt.normalize()
full_activity_df.head()
full_activity_df = full_activity_df.set_index('datetime')
full_activity_df.info()

NameError: name 'data' is not defined

# Join activity with sleep score

In [12]:
df_next = df['score'] # Create another subset of data to compare
df_next = df_next.shift(periods=-1, axis=0, fill_value=0) # Shift back by one (compare days data with next days score)
full_activity_df = full_activity_df.join(df_next, lsuffix='_activity', rsuffix='_sleep') # Join to original df with a new name

# Cleanup the data a bit by dropping NaN and 0 values
full_activity_df = full_activity_df.dropna(subset=['score_sleep'])

full_activity_df.head()



NameError: name 'df' is not defined

In [11]:
#corr = full_activity_df.corr()
#corr.style.background_gradient(cmap='coolwarm')
#corr.style.background_gradient(cmap='coolwarm').set_precision(2)


corr = full_activity_df.corr()['score_sleep'].sort_values()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(corr)


NameError: name 'full_activity_df' is not defined